<a href="https://colab.research.google.com/github/inuinana/pytorch_practice/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Akihiro Inui 2019 June
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # (input channel, output channel, kernel size)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # (input channel, output channel, kernel size)
        self.mp = nn.MaxPool2d(2) # Max pooling with kernel size 2x2
        self.fully_connected = nn.Linear(320, 10)
        self.optimizer = optim.SGD(self.parameters(), lr=0.01, momentum=0.5)
        
    def forward(self, x):
        # Input size
        in_size = x.size(0)
        # First conv layer with max pooling
        x = F.relu(self.mp(self.conv1(x)))
        # Second conv layer with max pooling
        x = F.relu(self.mp(self.conv2(x)))
        # Flatten image
        x = x.view(in_size, -1)  # flatten the tensor
        # Fully connected layer
        x = self.fully_connected(x)
        return F.log_softmax(x)


    def train(self, epoch):
        # Iteration for batch
        for batch_idx, (data, target) in enumerate(train_loader):
          
            # Split training dataset into data and label
            data, target = Variable(data), Variable(target)
            
            # Make prediction
            output = self(data)
            
            # Zero gradients, perform a backward pass, and update the weights.
            self.optimizer.zero_grad()
            self.loss = F.nll_loss(output, target)
            self.loss.backward()
            self.optimizer.step()
            
            # Print training status
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), self.loss.data))


    def test(self):
        self.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            # Split test dataset into data and label
            data, target = Variable(data, volatile=True), Variable(target)
            
            # Make prediction
            output = self(data)
            
            # Sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).data
            
            # Get the index of the max log-probability (predicted class)
            pred = output.data.max(1, keepdim=True)[1]
            
            # Count correct predictions
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

# main
if __name__ == "__main__":
    # Initialize CNN model
    model = CNN()
    
    # Training model
    for epoch in range(1, 10):
        model.train(epoch)
        
    # Test model
    model.test()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302543
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.289499
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.284662
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.268724
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.270043
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.264068
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.244671
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.226595
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.176491
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.158260
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.045141
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.960894
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.800942
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.656919
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.535902
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.145720
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.037771
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.021828
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.666448
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.593328
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0483, Accuracy: 9848/10000 (98%)

